In [1]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))


True
12.1
NVIDIA GeForce RTX 4050 Laptop GPU


In [1]:
import pandas as pd
import numpy as np

In [ ]:
# Path to your cleaned giant file
FILE_PATH = "prepared_data/preprocessed_full_data.csv"

# Columns we actually need for deep learning
keep_cols = [
    "timestamp", "detector_id", "congestion_index",
    "hour", "day_of_week", "month", "is_weekend",
    "is_holiday", "is_school_holiday", "is_rush_hour",
    "temperature", "precipitation", "visibility", "is_snow", "is_fog"
]

print("Loading only required columns...")
df = pd.read_csv(FILE_PATH, usecols=keep_cols)

# Memory reduction
for col in df.select_dtypes("float64"):
    df[col] = df[col].astype("float32")

for col in df.select_dtypes("int64"):
    df[col] = df[col].astype("int32")

#df["detector_id"] = df["detector_id"].astype("category")
df["timestamp"] = pd.to_datetime(df["timestamp"])

print(df.info())

Loading only required columns...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23385313 entries, 0 to 23385312
Data columns (total 15 columns):
 #   Column             Dtype         
---  ------             -----         
 0   timestamp          datetime64[ns]
 1   detector_id        category      
 2   hour               int32         
 3   day_of_week        int32         
 4   is_weekend         int32         
 5   month              int32         
 6   is_holiday         int32         
 7   is_rush_hour       int32         
 8   is_school_holiday  int32         
 9   congestion_index   float32       
 10  temperature        float32       
 11  precipitation      float32       
 12  visibility         float32       
 13  is_snow            int32         
 14  is_fog             int32         
dtypes: category(1), datetime64[ns](1), float32(4), int32(9)
memory usage: 1.4 GB
None


In [3]:
# Cyclical encoding of time features
df["hour_sin"]  = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"]  = np.cos(2 * np.pi * df["hour"] / 24)

df["dow_sin"]   = np.sin(2 * np.pi * df["day_of_week"] / 7)
df["dow_cos"]   = np.cos(2 * np.pi * df["day_of_week"] / 7)

df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

In [ ]:
# Pick a few detectors for fast experimenting
#sample_detectors = df["detector_id"].cat.categories[:3]  # first n detectors
sample_detectors = df["detector_id"].unique()[:3]  # first n detectors

df_small = df[df["detector_id"].isin(sample_detectors)]
#df_small["detector_id"] = df_small["detector_id"].cat.remove_unused_categories()


# Restrict to 2019-2020 for prototyping
df_small = df_small[df_small["timestamp"].dt.year.isin([2022, 2020, 2021])]

df_small = df_small.sort_values(["detector_id", "timestamp"]).reset_index(drop=True)

print("Small dataframe size:", df_small.shape)

Small dataframe size: (40794, 21)


In [6]:
df_small["detector_id"].unique()
df_small.head()

,timestamp,detector_id,hour,day_of_week,is_weekend,month,is_holiday,is_rush_hour,is_school_holiday,congestion_index,...,precipitation,visibility,is_snow,is_fog,hour_sin,hour_cos,dow_sin,dow_cos,month_sin,month_cos
0,2020-01-01 00:00:00,-1792767705,0,2,0,1,0,0,1,0.471233,...,0.0,5000.0,0,0,0.000000,1.000000,0.974928,-0.222521,0.5,0.866025
1,2020-01-01 02:00:00,-1792767705,2,2,0,1,0,0,1,0.500685,...,0.0,10000.0,0,0,0.500000,0.866025,0.974928,-0.222521,0.5,0.866025
2,2020-01-01 03:00:00,-1792767705,3,2,0,1,0,0,1,0.497945,...,0.0,9000.0,0,0,0.707107,0.707107,0.974928,-0.222521,0.5,0.866025
3,2020-01-01 04:00:00,-1792767705,4,2,0,1,0,0,1,0.482877,...,0.0,7000.0,0,0,0.866025,0.500000,0.974928,-0.222521,0.5,0.866025
4,2020-01-01 05:00:00,-1792767705,5,2,0,1,0,0,1,0.467123,...,0.0,6000.0,0,0,0.965926,0.258819,0.974928,-0.222521,0.5,0.866025


In [6]:
# Add lags

def make_lags(df, col, lags):
    for lag in lags:
        df[f"{col}_lag_{lag}h"] = df.groupby("detector_id")[col].shift(lag)
    return df


weather_lags = [-i for i in range(1, 25, 8)] # Next 24 hours
df_small = make_lags(df_small, "temperature", weather_lags)
df_small = make_lags(df_small, "precipitation", weather_lags)
df_small = make_lags(df_small, "visibility", weather_lags)

# Remove rows with NaNs due to lagging
df_small = df_small.dropna().reset_index(drop=True)


C:\Users\adib4\AppData\Local\Temp\ipykernel_35388\3933638425.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"{col}_lag_{lag}h"] = df.groupby("detector_id")[col].shift(lag)
C:\Users\adib4\AppData\Local\Temp\ipykernel_35388\3933638425.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df[f"{col}_lag_{lag}h"] = df.groupby("detector_id")[col].shift(lag)
C:\Users\adib4\AppData\Local\Temp\ipykernel_35388\3933638425.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to ad

In [ ]:
from sklearn.preprocessing import StandardScaler

# -----------------------------
# 1) Define parameters
# -----------------------------
HISTORY_OFFSETS = [0, 1, 3, 6, 12, 18, 24, 48]   # hours before t
forecast_horizon = 24      # next 24 hours → target

feature_cols_norm = [
    "temperature", "precipitation", "visibility", "congestion_index"
]  + [f"temperature_lag_{lag}h" 
    for lag in weather_lags] + [f"precipitation_lag_{lag}h" 
    for lag in weather_lags] + [f"visibility_lag_{lag}h" for lag in weather_lags]

feature_cols = [
    "hour_sin", "hour_cos", "dow_sin", "dow_cos", "month_sin", "month_cos",
    "is_weekend", "is_holiday", "is_school_holiday", "is_rush_hour",
    "temperature", "precipitation", "visibility", "is_snow", "is_fog",
    "congestion_index"
] + [f"temperature_lag_{lag}h" 
    for lag in weather_lags] + [f"precipitation_lag_{lag}h" 
    for lag in weather_lags] + [f"visibility_lag_{lag}h" for lag in weather_lags]


# -----------------------------
# 2) Train / Val / Test split
# -----------------------------
train = df_small[df_small["timestamp"] < "2021-05-01"].copy()
val   = df_small[(df_small["timestamp"] >= "2021-05-01") &
                 (df_small["timestamp"] <  "2022-01-01")].copy()
test  = df_small[df_small["timestamp"] >= "2022-01-01"].copy()

print(train.shape, val.shape, test.shape)

# -----------------------------
# 3) Normalize continuous features
# -----------------------------
scaler = StandardScaler()
train[feature_cols_norm] = scaler.fit_transform(train[feature_cols_norm])
val[feature_cols_norm]   = scaler.transform(val[feature_cols_norm])
test[feature_cols_norm]  = scaler.transform(test[feature_cols_norm])

(125883, 30) (21549, 30) (18788, 30)


In [ ]:
# -----------------------------
# 4) Function to create sequences
# -----------------------------
def create_nhits_sequences(df, feature_cols, hist_offsets, horizon):
    X_list, Y_list, idx_list = [], [], []

    for det_id, df_det in df.groupby("detector_id"):
        df_det = df_det.sort_values("timestamp").reset_index(drop=False)
        #   keep original row index      ↑↑↑

        values = df_det[feature_cols].values.astype(np.float32)
        target = df_det["congestion_index"].values.astype(np.float32)
        idx    = df_det["index"].values    # original global index

        n = len(df_det)
        for t in range(max(hist_offsets), n - horizon):
            X_list.append(values[[t-h for h in hist_offsets]])
            Y_list.append(target[t : t+horizon])
            idx_list.append(idx[t])   # <--- store index where prediction starts

    return (
        np.array(X_list, dtype=np.float32),
        np.array(Y_list, dtype=np.float32),
        np.array(idx_list, dtype=np.int64),
    )


# -----------------------------
# 5) Choose the model input features
# -----------------------------
model_features = feature_cols

# -----------------------------
# 6) Build sequences for each split
# -----------------------------
X_train_hist, Y_train, train_idx = create_nhits_sequences(
    train, model_features, HISTORY_OFFSETS, forecast_horizon)

X_val_hist, Y_val, val_idx = create_nhits_sequences(
    val, model_features, HISTORY_OFFSETS, forecast_horizon)

X_test_hist, Y_test, test_idx = create_nhits_sequences(
    test, model_features, HISTORY_OFFSETS, forecast_horizon)

print("Shapes:")
print("X_train_hist:", X_train_hist.shape)
print("Y_train:", Y_train.shape)
print("train_idx:", train_idx.shape)
print("X_val_hist:",   X_val_hist.shape)
print("Y_val:",   Y_val.shape)
print("val_idx:", val_idx.shape)
print("X_test_hist:",  X_test_hist.shape)
print("Y_test:",  Y_test.shape)
print("test_idx:", test_idx.shape)

C:\Users\adib4\AppData\Local\Temp\ipykernel_35388\2960479339.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for det_id, df_det in df.groupby("detector_id"):
C:\Users\adib4\AppData\Local\Temp\ipykernel_35388\2960479339.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for det_id, df_det in df.groupby("detector_id"):
C:\Users\adib4\AppData\Local\Temp\ipykernel_35388\2960479339.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence thi

Shapes:
X_train_hist: (125163, 8, 25)
Y_train: (125163, 24)
X_val_hist: (20829, 8, 25)
Y_val: (20829, 24)
X_test_hist: (18068, 8, 25)
Y_test: (18068, 24)


In [9]:
import torch
from torch.utils.data import Dataset

class NHitsDataset(Dataset):
    def __init__(self, X_hist, Y):
        self.X_hist = X_hist
        self.Y = Y

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X_hist[idx]).float(),
            torch.from_numpy(self.Y[idx]).float(),
        )



In [ ]:
from torch.utils.data import DataLoader

batch_size = 512     # can increase if GPU is large

train_loader = DataLoader(
    NHitsDataset(X_train_hist, Y_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    drop_last=True
)

val_loader = DataLoader(
    NHitsDataset(X_val_hist, Y_val),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

test_loader = DataLoader(
    NHitsDataset(X_test_hist, Y_test),
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)


In [11]:
import torch.nn as nn

class MLPForecaster(nn.Module):
    def __init__(self, input_length, num_features, horizon):
        super().__init__()
        in_dim = input_length * num_features

        self.net = nn.Sequential(
            nn.Linear(in_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, horizon)
        )

    def forward(self, x):
        # Flatten sequence into one vector per sample
        x = x.reshape(x.size(0), -1)
        return self.net(x)


In [ ]:
from tqdm import tqdm

def train(model, train_loader, val_loader, criterion, optimizer, scaler, device, num_epochs):
    model.train()
    train_losses = []
    val_losses = []
    for epoch in tqdm(range(num_epochs)):
        loss_epoch = 0
        for X_batch, Y_batch in tqdm(train_loader):
            X_batch = X_batch.to(device, non_blocking=True)
            Y_batch = Y_batch.to(device, non_blocking=True)

            optimizer.zero_grad()

            with torch.amp.autocast("cuda"):
                preds = model(X_batch)
                loss = criterion(preds, Y_batch)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            loss_epoch += loss.item()

        train_losses.append(loss_epoch / len(train_loader))

        model.eval()
        val_loss = 0

        with torch.no_grad():
            for X_batch, Y_batch in tqdm(val_loader):
                X_batch = X_batch.to(device, non_blocking=True)
                Y_batch = Y_batch.to(device, non_blocking=True)
                preds = model(X_batch)
                loss = criterion(preds, Y_batch)
                val_loss += loss.item()
        val_losses.append(val_loss / len(val_loader))
        
        tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_losses[-1]:.4f} - Val Loss: {val_losses[-1]:.4f}")

    return train_losses, val_losses

    
def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    preds_list = []

    with torch.no_grad():
        for X_batch, Y_batch in tqdm(loader):
            X_batch = X_batch.to(device, non_blocking=True)
            Y_batch = Y_batch.to(device, non_blocking=True)
            preds = model(X_batch)
            loss = criterion(preds, Y_batch)
            total_loss += loss.item()
            preds_list.append(preds.cpu())

    preds_list = torch.cat(preds_list, dim=0)
    return preds_list, total_loss / len(loader)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = MLPForecaster(
    input_length=len(HISTORY_OFFSETS),
    num_features=X_train_hist.shape[-1],
    horizon=forecast_horizon
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()
scaler = torch.amp.GradScaler("cuda")  # AMP = automatic mixed precision

epochs = 20

train_losses, val_losses = train(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scaler,
    device,
    epochs
)

model.eval()
preds, test_loss = evaluate(model, test_loader, criterion, device)
print("Test loss:", test_loss)


C:\Users\adib4\AppData\Local\Temp\ipykernel_35388\2950684459.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()   # AMP = automatic mixed precision
  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
import numpy as np

def evaluate_block_predictions(Y_true, Y_pred):
    """
    Ultra-fast evaluation of block forecasts using vectorized operations.
    Shape: Y_true, Y_pred = (N, H)
    """

    # ---- Basic metrics (vectorized) ----
    diff = Y_true - Y_pred
    mae  = np.mean(np.abs(diff))
    rmse = np.sqrt(np.mean(diff * diff))
    print("MAE:", mae, "RMSE:", rmse)

    # ---- Vectorized correlation ----
    # Center sequences
    Yt = Y_true - Y_true.mean(axis=1, keepdims=True)
    Yp = Y_pred - Y_pred.mean(axis=1, keepdims=True)
    print("Mean centered Y_true and Y_pred for correlation calculation.")

    # Compute numerator: cov
    num = np.sum(Yt * Yp, axis=1)

    # Denominator: std(True)*std(Pred)
    denom = np.sqrt(np.sum(Yt * Yt, axis=1) * np.sum(Yp * Yp, axis=1))
    print("Computed denominator for correlation calculation.")

    # Avoid division by zero
    corr_per_block = np.where(denom == 0, np.nan, num / denom)

    # Average correlation (ignoring NaN)
    corr = np.nanmean(corr_per_block)
    print("Correlation:", corr)

    return {
        "MAE": mae,
        "RMSE": rmse,
        "Corr": corr
    }


In [ ]:
import matplotlib.pyplot as plt

def plot_block_predictions(df, horizon=24, detector_id=None,
                           years=None, months=None,
                           true_prefix="future_", pred_prefix="pred_",
                           max_blocks=10):
    """
    Plot 24h forecast trajectories:
    - t+1 ... t+horizon for each chosen block
    - sample every 'horizon' timestamps
    """

    df_plot = df.copy()

    # ---- FILTERING ----
    if detector_id is None:
        detector_id = df_plot["detector_id"].iloc[0]
    df_plot = df_plot[df_plot["detector_id"] == detector_id]

    if years is not None:
        df_plot = df_plot[df_plot["timestamp"].dt.year.isin(years)]

    if months is not None:
        df_plot = df_plot[df_plot["timestamp"].dt.month.isin(months)]

    # ---- Take blocks every 'horizon' timesteps ----
    df_blocks = df_plot.iloc[::horizon].copy()
    df_blocks = df_blocks.head(max_blocks)

    # ---- Column lists ----
    true_cols = [f"{true_prefix}{h}h" for h in range(1, horizon+1)]
    pred_cols = [f"{pred_prefix}{h}h" for h in range(1, horizon+1)]

    plt.figure(figsize=(14, 7))
    print("Plotting...")
    for _, row in df_blocks.iterrows():
        base_time = row["timestamp"]
        horizon_times = base_time + pd.to_timedelta(np.arange(1, horizon+1), "h")

        plt.plot(horizon_times, row[true_cols].values,
                 label=f"True (start {base_time})", alpha=0.6)

        plt.plot(horizon_times, row[pred_cols].values,
                 label=f"Pred (start {base_time})", alpha=0.6)

    plt.title(f"{horizon}-hour Forecast Trajectories")
    plt.xlabel("Time")
    plt.ylabel("Congestion Index")
    plt.legend()
    plt.show()


In [ ]:
def evaluate_and_plot_block(df, horizon=24,
                            detector_id=None,
                            years=None,
                            months=None,
                            true_prefix="future_",
                            pred_prefix="pred_"):
    """
    Full multi-step forecast evaluation and plotting.
    """

    # ---- Extract arrays ----
    true_cols = [f"{true_prefix}{h}h" for h in range(1, horizon+1)]
    pred_cols = [f"{pred_prefix}{h}h" for h in range(1, horizon+1)]

    Y_true = df[true_cols].values
    Y_pred = df[pred_cols].values

    # ---- Compute metrics ----
    metrics = evaluate_block_predictions(Y_true, Y_pred)

    print("=== Block Forecast Evaluation ===")
    for k, v in metrics.items():
        print(f"{k}: {v:.4f}")

    # ---- Plot ----
    plot_block_predictions(
        df,
        horizon=horizon,
        detector_id=detector_id,
        years=years,
        months=months,
        true_prefix=true_prefix,
        pred_prefix=pred_prefix
    )

    return metrics


In [ ]:
def historical_baseline_multi(df, window_size=5, horizon=24):
    df_h = df[["detector_id", "timestamp", "congestion_index"]].copy()

    # For each horizon h = 1..24 create a future target
    for h in range(1, horizon+1):
        df_h[f"future_{h}h"] = (
            df_h.groupby("detector_id")["congestion_index"]
                .shift(-h)
        )

    # Baseline uses rolling mean of past
    df_h["hist_baseline"] = (
        df_h.groupby("detector_id")["congestion_index"]
             .rolling(window_size, min_periods=1)
             .mean()
             .reset_index(level=0, drop=True)
    )

    # Expand baseline into 24 identical horizons
    for h in range(1, horizon+1):
        df_h[f"pred_{h}h"] = df_h["hist_baseline"]

    # Drop rows where ANY future target is missing
    future_cols = [f"future_{h}h" for h in range(1, horizon+1)]
    df_h = df_h.dropna(subset=future_cols)

    return df_h


print("Baseline: historical average congestion.")
df_historical = historical_baseline_multi(test, horizon=24)
evaluate_and_plot_block(df_historical, horizon=24, years=[2022])


In [ ]:
print("DL Model Evaluation on Test Set")

eval_df = pd.DataFrame({
    "row_idx": test_idx,
    "timestamp": test.loc[test_idx, "timestamp"].values,
    "detector_id": test.loc[test_idx, "detector_id"].values
})

for h in range(1, forecast_horizon+1):
    eval_df[f"pred_{h}h"] = preds[:, h-1].numpy()

for h in range(1, forecast_horizon+1):
    eval_df[f"future_{h}h"] = (
        test.groupby("detector_id")["congestion_index"]
            .shift(-h)
            .loc[test_idx]
            .values
    )

eval_df = eval_df.dropna()


evaluate_and_plot_block(eval_df, horizon=24, years=[2022])